In [70]:
# Libraries
import quandl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader.data as data
import datetime
import json
from yahoofinancials import YahooFinancials

asset_dict = {}

# Assets
russell_index = ["^RUI"]
tickers = []

# Set timeframe
start = "2019-03-31"
end = "2019-04-30"

def russell():
    # Pull data from Yahoo! Finance for Russell 1000 Index
    df = data.DataReader(russell_index, data_source="yahoo", start=start, end=end)

    percentage = df.pct_change()
    russellReturn = percentage["Adj Close"]

    return russellReturn


#russell()
def pullTickers():
    exec(open("scrapeETF.py").read())

    list(asset_dict.keys())

    for key in asset_dict.keys():
        tickers.append(key)
    return tickers

pullTickers()




Looking for [chromedriver 79.0.3945.36 win32] driver in cache
File found in cache by path [C:\Users\lukes\.wdm\drivers\chromedriver\79.0.3945.36\win32\chromedriver.exe]


['AAPL',
 'MSFT',
 'AMZN',
 'FB',
 'BRK.B',
 'JPM',
 'GOOG',
 'GOOGL',
 'JNJ',
 'V',
 'PG',
 'XOM',
 'BAC',
 'T',
 'UNH',
 'MA',
 'INTC',
 'VZ',
 'DIS',
 'HD',
 'MRK',
 'CVX',
 'PFE',
 'KO',
 'CSCO']

In [65]:
yahoo_financials = YahooFinancials('msft')
print(yahoo_financials.get_financial_stmts('quarterly', 'balance')['balanceSheetHistoryQuarterly']['MSFT'][2]['2019-03-31'])

{'intangibleAssets': 8103000000, 'totalLiab': 168417000000, 'totalStockholderEquity': 94864000000, 'otherCurrentLiab': 32295000000, 'totalAssets': 263281000000, 'commonStock': 77791000000, 'otherCurrentAssets': 7131000000, 'retainedEarnings': 18338000000, 'otherLiab': 36471000000, 'goodWill': 41861000000, 'treasuryStock': -1265000000, 'otherAssets': 10258000000, 'cash': 11212000000, 'totalCurrentLiabilities': 53861000000, 'shortLongTermDebt': 6515000000, 'otherStockholderEquity': -1265000000, 'propertyPlantEquipment': 40769000000, 'totalCurrentAssets': 159887000000, 'longTermInvestments': 2403000000, 'netTangibleAssets': 44900000000, 'shortTermInvestments': 120324000000, 'netReceivables': 19269000000, 'longTermDebt': 66585000000, 'inventory': 1951000000, 'accountsPayable': 7544000000}


In [ ]:
def buildPortfolio():
    











